Del 1 – Grundläggande analys och statistik (Hälsostudie)

I denna notebook genomför jag en grundläggande analys av hälsodatasetet. 
Jag använder Python för att skapa beskrivande statistik, visualiseringar, 
simuleringar samt hypotest enligt instruktionerna i uppgiften.


In [ ]:
# Import och dataimport

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind


In [ ]:
file_path = "health_study_dataset.csv"
df = pd.read_csv(file_path)


In [ ]:
%matplotlib inline

pd.options.display.max_rows = 20
pd.options.display.float_format = "{:,.2f}".format


Vi börjar med att titta på de första fem raderna i datasetet. Det ger en snabb överblick över kolumnerna: ålder, kön, längd, vikt, blodtryck, kolesterol, rökstatus och sjukdom.

In [ ]:
df.head()

In [ ]:
## Grundläggade information och null_värden 

df.info()

Datasetet innehåller 800 observationer och inga saknade värden i någon kolumn.


Beskrevande statistik


In [ ]:
## Statistisk sammanfattning

df.describe()

Här ser vi grundläggande statistik för numeriska kolumner:

medelåder: 45,93 år
medelvikt 71,41 kg
medelvärde för systoliskt blodtryck: 141,18 mmHg
endast 6% av deltagarna har sjukdomen


In [ ]:
## Könsfördelning och rökstatus

df["sex"].value_counts()

Fördelningen mellan kön är nästan jämn:

Kvinnor (F) 403
Män (M) 397


In [ ]:
## Andel rökare

df["smoker"].value_counts()

Rökstatus i datasetet:

Icke-rökare: (No) 587 personer (73.4%)
Rökare (Yes): 213 personer (26.6%)


In [ ]:
## Samband mellan längd och vikt

df[["height","weight"]].corr()

Korrelationen mellan längd och vikt är 0.32, vilket tyder på ett svagt positvivt samband- längre personer tenderar att väga mer men inte starkt.


In [ ]:
## Standardavvikelse för nyckelvariabler

df[["height", "weight", "systolic_bp", "cholesterol"]].std()

Standardavvikelsen visar hur mycket värdena varierar kring medelvärdet. till exempel har systoliskt blodtryck en relativt hög spridning, vilken kan påverka analysen.


In [ ]:
## Sammanfattande statistik för hälsovariabler

df[["age", "height", "weight", "systolic_bp", "cholesterol"]].agg(["mean", "median", "min", "max"])

Vi beräknar fyra centrala mått för varje variabel:

Medelvärde (mean): Genomsnittligt värde
Median: Mittenvärdet när data sorteras
Min: Lägsta observerade värde
Max: Högsta observerade värde
Detta ger en snabb överblick över både centraltendens och spridning. Till exempel:

Om medelvärde och median skiljer sig mycket → kan tyda på snedfördelning
Om max är mycket högre än median → kan finnas outliers
Detta steg är viktigt innan vi går vidare till visualisering och hypotesprövning.
    


Group analysis


In [ ]:
## Genomsnittliga värden per kön

df.groupby("sex")[["height", "weight", "systolic_bp", "cholesterol"]].mean()

Jag grupper data efter kön och beräknar medelvärden för längd, vikt, blodtrygg och kolesterol. Resultatrt visar:

Kvinor (F) genomsnittlig
- Längd: 165,10 cm
- Vikt : 75.92 kg
- Blodtryck: 148.13 mmHg
- Kolesterolvärde: 4,85
Män (M) genomsnittlig
- Längd: 178.70 cm
- Vikt: 75,92
- Blodtryck: 150,24
- Kolesterolvärde: 4,97
Män är längre och har högre blodtryck.


In [ ]:
## Genomsnittligt kolesterolvärde per rökstatus

df.groupby("smoker")["cholesterol"].mean()

Jag grupperar data efter rökstatus och beröknar genomsnittligt kolesterolvärde för varje grupp:

- Icke-rökare (No): 4,91
- Rökare: 5.01


In [ ]:
## Skapa BMI-kolumn

df["bmi"] = df["weight"] / (df["height"]/100)**2
df["bmi"].describe()

BMI är ett vanligt hälsomått som kan användas i vidare analys, t.ex. för att undersöka samband med blodtryck eller sjukdom.

Visualiseringar


In [ ]:
## Histogram över systoliskt blodtryck

plt.hist(df["systolic_bp"], bins=20, color="green")
plt.xlabel("Systiolic Blood Pressure")
plt.ylabel("Antal")
plt.title("Histogram över systoliskt blodtryck")
plt.show()

Detta histogram visar fördelningen av systoliskt blodtryck i populationen. Varje stapel representerar antalet personer vars blodtryck faller inom ett visst intervall.

- Histogrammet använder 20 bins, vilket ger en detaljerad bild av fördelningen.
- Färgen green gör diagrammet lättläst och visuellt tilltalande.
- Vi kan se om fördelningen är symmetrisk, sned, eller om det finns outliers.
Om fördelningen är tydligt sned (t.ex. högerförskjuten) kan det påverka valet av statistiska tester i nästa steg.

In [ ]:
## Boxplot: vikt per kön

df.boxplot(column="weight", by="sex", figsize=(6,6))
plt.suptitle("")
plt.title("Boxplot: Vikt per kön")
plt.xlabel("Vikt (kg)")
plt.show()

Diagrammet visar fördelningen av kroppsvikt mellan olika kön. Varje box representerar den centrala 50% av värdena (interkvartilintervall), med medianen som ett horisontellt streck i mitten. Prickar utanför boxen är potentiella outliers.

- Kvinnor (F) tenderar att ha lägre medianvikt än män.
- Män (M) har en bredare spridning, vilket kan tyda på större variation i kroppsvikt.
Boxploten ger en visuell överblick som kompletterar den statistiska sammanfattningen i tidigare steg. Den är särskilt användbar inför hypotesprövning eller när vi vill förstå gruppskillnader.



In [ ]:
## Scatterplot: samband mellan längd och vikt

sns.scatterplot(x="height", y="weight", hue="sex", data=df)
plt.title("Samband mellan längd och vikt")
plt.xlabel("Längd (cm)")
plt.ylabel("Vikt (kg)")
plt.show()

Diagrammet visar sambandet mellan längd och vikt i datasetet. Varje punkt representerar 
en individ och färgen markerar kön.

- Det finns ett tydligt positivt samband: längre personer tenderar att väga mer.
- Spridningen är relativt stor inom båda könsgrupperna, vilket tyder på att andra faktorer 
  än längd också påverkar vikten.
- Män (orange) ligger generellt högre både i längd och vikt.

Scatterploten hjälper oss att visuellt förstå korrelationen och eventuell gruppskillnad.


In [ ]:
## Stapeldiagram: Antal rökare vs icke-rökare

df["smoker"].value_counts().plot(kind="bar", color=["green", "red"])
plt.title("Antal rökare vs Icke-rökare")
plt.xlabel("Rökstatus")
plt.ylabel("Antal personer")
plt.show()


Diagrammet visar hur många personer i studien som är rökare respektive icke-rökare.

- Grön stapel: representerar icke-rökare (No)
- Röd stapel: representerar rökare (Yes)
- Diagrammet ger en snabb överblick över gruppstorlek, vilket är viktigt inför statistisk analys

Om grupperna är mycket olika i storlek kan det påverka tolkningen av medelvärden och resultat från hypotesprövning.


Simulering av sjukdom (disease)

Syftet här är att uppskatta den observerade andelen sjuka i datasetet och sedan jämföra detta med 1000 simulerade stickprov som utgår från samma sannolikhet (p = andel sjuka).


In [ ]:
p_disease = df ["disease"].mean()
p_disease

In [ ]:
np.random.seed(42) #Ger reproducerbara resultat

n = len(df) # Storleken på det riktiga stickprovet
n_sim = 1000 # Antal simuleringar 

# Binomial slumpninh: antal sjuka per simulering
simulated_counts = np.random.binomial(n=n, p=p_disease, size=n_sim)

# Konvertera till andelar 
simulated_props = simulated_counts / n

# Visa några av de simulerade värdena
simulated_props[:10]

In [ ]:
sim_mean = simulated_props.mean()
sim_std = simulated_props.std()

sim_mean, sim_std

In [ ]:
plt.figure(figsize=(8,5))
plt.hist(simulated_props, bins=30, color="skyblue", edgecolor="black")
plt.axvline(p_disease, color="red", linestyle="--", linewidth=2, label="Observerad andel")
plt.xlabel("Andel sjuka (simulerad)")
plt.ylabel("Antal simuleringar")
plt.title("Simulerad andel sjuka jämför med observerad andel")
plt.legend()
plt.show()

Den observerade andelen sjuka i datasetet är p_disease (≈ X %).
I 1000 simuleringar med samma sannolikhet fick jag:
- Medelvärde: sim_mean
- Standardavvikelse: sim_std
Den observerade andelen ligger nära mitten av fördelningen av de simulerade värdena, vilket tyder på att den observerade andelen sjuka är rimlig givet den antagna sannolikheten. Detta innebär att modellen med en fast sannolikhet för sjukdom fungerar väl i detta fall.


Hypotest: Har rökare högre systoliskt blodtryck än icke-rökare?

Syftet här är att testa om rökare har högre genomsnittligt systoliskt blodtryck (systolic_bp) jämfört med icke-rökare.

Nollhypotes (H0):
μ_rökare ≤ μ_icke-rökare
(Rökare har inte högre blodtryck.)

Alternativ hypotes (H1): μ_rökare > μ_icke-rökare
(Rökare har högre blodtryck.)

Eftersom vi jämför två oberoende grupper använder vi ett ensidigt t-test (independent samples t-test).

In [ ]:
# Filtrera grupper

smokers = df[df["smoker"] == "Yes"]["systolic_bp"].dropna()
nonsmokers = df[df["smoker"] == "No"]["systolic_bp"].dropna()

# Visa medelväeden för känsla

smokers.mean(), nonsmokers.mean(), len(smokers), len(nonsmokers)

In [ ]:
t_stat, p_two_sided = ttest_ind(smokers, nonsmokers, equal_var=False)
t_stat, p_two_sided

In [ ]:
# Ensidigt test (H1: rökare > icke-rökare)

mean_smok = smokers.mean()
mean_nonsmok = nonsmokers.mean()

if mean_smok > mean_nonsmok:
    p_one_sided = p_two_sided / 2
else:
    p_one_sided = 1 - (p_two_sided / 2)

mean_smok, mean_nonsmok, p_one_sided

Resultat av hypotestet
- Medelvärde systolic_bp för rökare: X mmHg
- Medelvärde systolic_bp för icke-rökare: Y mmHg
- Ensidigt p-värde: p_one_sided
Tolkning:
- Om p < 0,05 förkastar jag nollhypotesen och det finns evidens för att rökare har högre systoliskt blodtryck än icke-rökare i detta dataset.
- Om p ≥ 0,05 kan jag inte förkasta nollhypotesen och det finns inte tillräckligt med stöd för att säga att rökare har högre blodtryck.
(Sätt in de faktiska värdena som genereras i koden ovan.)


Sammanfattning

I denna analys undersökte jag datasetet med 800 deltagare och analyserade
grundläggande statistik, gruppskillnader, samband och en hypotesprövning.

De viktigaste resultaten:

- Deltagarna hade i snitt ett systoliskt blodtryck på ca 141 mmHg och ett 
  kolesterolvärde runt 5 mmol/L.
- Kvinnor och män skiljde sig åt i längd och blodtryck, men hade liknande vikt.
- Korrelationsanalysen visade ett svagt positivt samband mellan längd och vikt.
- Andelen sjuka i populationen var ca 6 %, och simuleringar visade att den
  observerade andelen ligger väl inom den slumpmässiga variationen.
- Hypotestet för att avgöra om rökare har högre systoliskt blodtryck gav 
  (signifikant / inte signifikant) resultat vid 5%-nivån.

Analysen visar att det finns vissa mönster i datan, men flera av skillnaderna 
är små och bör tolkas med försiktighet. Ytterligare analys, exempelvis fler 
variabler eller regressionsmodeller, skulle kunna ge en djupare förståelse.
